In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import tree
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import glob
from sklearn.feature_selection import mutual_info_classif as MIC
from matplotlib import pyplot
from sklearn.metrics import accuracy_score
import time

In [2]:
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import GridSearchCV
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.base import BaseEstimator
from sklearn.base import ClassifierMixin
from sklearn.preprocessing import LabelEncoder
import six
import sys
sys.modules['sklearn.externals.six'] = six
from sklearn.externals import six
from sklearn.base import clone
from sklearn.pipeline import _name_estimators
from sklearn.ensemble import RandomForestClassifier

In [3]:
# number = int(get_ipython().user_ns['param'])
number = 1

In [4]:
model_acc=[]
model_precision=[]
model_recal=[]
model_f1=[]
model_nam=[]
model_confusion=[]
model_time=[]
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report,f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score, confusion_matrix

In [5]:
df = pd.read_csv('../Dataset CSV/csv_result-data'+str(number)+' Sampled Scenarios.csv')

In [6]:
# Perform the operation to create the new feature
df['R1-VCM1'] = (df['R1-PM1:V'] - df['R1-PM7:V']) / (df['R1-PM4:I'] - df['R1-PM10:I'])
df['R3-sV1'] = np.sin(df['R3-PA1:VH'] - df['R3-PA7:VH'])
df['R3-sV3'] = np.sin(df['R3-PA3:VH'] - df['R3-PA9:VH'])
# df['R2-VCM2'] = (df['R2-PM2:V'] - df['R2-PM8:V']) / (df['R2-PM5:I'] - df['R2-PM11:I'])
df['R2-VCM1'] = (df['R2-PM1:V'] - df['R2-PM7:V']) / (df['R2-PM4:I'] - df['R2-PM10:I'])
df['R3-VCM1'] = (df['R3-PM1:V'] - df['R3-PM7:V']) / (df['R3-PM4:I'] - df['R3-PM10:I'])
df['R4-sI1'] = np.sin(df['R4-PA4:IH'] - df['R4-PA10:IH'])
df['R2-VCA4'] = df['R2-PA7:VH'] - df['R2-PA10:IH']
df['R1-sI1'] = np.sin(df['R1-PA4:IH'] - df['R1-PA10:IH'])
df['R4-VCM2'] = (df['R4-PM2:V'] - df['R4-PM8:V']) / (df['R4-PM5:I'] - df['R4-PM11:I'])
df['R2-VCA1'] = np.sin(df['R2-PA1:VH'] - df['R2-PA4:IH'] - df['R2-PA7:VH'] - df['R2-PA10:IH'])
df['R3-VCM2'] = (df['R3-PM2:V'] - df['R3-PM8:V']) / (df['R3-PM5:I'] - df['R3-PM11:I'])
df['R1-VCA4'] = df['R1-PA7:VH'] - df['R1-PA10:IH']
df['R1-VCM2'] = (df['R1-PM2:V'] - df['R1-PM8:V']) / (df['R1-PM5:I'] - df['R1-PM11:I'])
df['R2-VCM2'] = (df['R2-PM2:V'] - df['R2-PM8:V']) / (df['R2-PM5:I'] - df['R2-PM11:I'])
df['R4-VCA4'] = df['R4-PA7:VH'] - df['R4-PA10:IH']
df['R4-VCM1'] = (df['R4-PM1:V'] - df['R4-PM7:V']) / (df['R4-PM4:I'] - df['R4-PM10:I'])

In [7]:
# add the marker to the last of the list
cols = list(df.columns)
cols.remove('marker')
df = df[cols + ['marker']]

In [8]:
df.replace([np.inf, -np.inf], np.nan, inplace= True)
df.isnull().sum().sum()

2951

In [9]:
from sklearn.impute import KNNImputer

# Create an instance of the KNNImputer class
imputer = KNNImputer(n_neighbors=1)

# Fit the imputer on the data
imputed_data = imputer.fit_transform(df)

# Replace the missing values in the original dataframe
df = pd.DataFrame(imputed_data, columns=df.columns)

In [10]:
print(df.isin([np.inf, -np.inf]).sum().sum(), df.isnull().sum().sum())

0 0


In [11]:
df_z_scaled = df.copy()
for column in df_z_scaled.columns:
#     print(df_z_scaled[column])
    if (df_z_scaled[column].std() != 0) and (column != 'id') and (column != 'marker'):
        df_z_scaled[column] = (df_z_scaled[column] -
                           df_z_scaled[column].mean()) / df_z_scaled[column].std()     
# print(df_z_scaled)

In [12]:
X = df_z_scaled.iloc[:,1:-1]
y = df_z_scaled.iloc[:,-1:]

In [13]:
# Label the marker data
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)
y = pd.DataFrame(y, columns=["marker"])

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [14]:
Get the names of the top 31 features
top_n_feature_names = ['R1-PM5:I', 'R1-VCM1', 'R3-PM5:I', 'R1-PA1:VH', 'R4-PA2:VH', 'R3-sV3',
 'R1-PA3:VH', 'R2-VCM1', 'R4-VCM1', 'R3-PA5:IH', 'R1-PA2:VH', 'R3-sV1',
 'R2-PA5:IH', 'R2-PA3:VH', 'R2-PA6:IH', 'R2-PA1:VH', 'R3-VCM1', 'R4-PA5:IH',
 'R2-VCA4', 'R3-PA6:IH', 'R4-PM2:V', 'R2-PA4:IH', 'R3-PA4:IH', 'R1-PA5:IH',
 'R1-sI1', 'R2-PA2:VH', 'R1-PM2:V', 'R3-PA:ZH', 'R2-VCM2', 'R1-VCM2']

# Filter the original dataframe to only include the top n features
X_top_n = X[top_n_feature_names].copy()

X_new = X_top_n.copy()

In [15]:
X_new

,R1-PM5:I,R3-PM5:I,R1-VCM1,R3-sV3,R1-PA1:VH,R4-VCM1,R1-PA2:VH,R3-PA5:IH,R2-PA1:VH,R2-PA5:IH,...,R1-PA:ZH,R2-PA6:IH,R2-VCA1,R4-PA:ZH,R3-PA6:IH,R3-PA4:IH,R2-PA4:IH,R4-PM2:V,R1-PA12:IH,R1-PM2:V
0,1.647816,1.568783,0.055094,0.348533,0.900357,-0.039146,-0.505742,1.330254,0.819485,1.313361,...,0.234719,0.136668,1.307423,0.233750,0.149757,-1.384384,-1.402208,-1.141293,-1.509542,-1.340787
1,0.768988,0.709833,0.071102,1.287898,0.933881,-0.069421,-0.477304,1.378097,0.874259,1.360758,...,0.007636,0.178502,-1.400366,0.046543,0.191498,-1.337799,-1.355229,-0.438748,-1.442046,-0.372331
2,0.768988,0.709833,0.071102,1.261446,0.934349,-0.045633,-0.476906,1.378715,0.874704,1.361465,...,0.020810,0.178452,-1.391531,0.044523,0.191550,-1.337626,-1.355173,-0.431992,-1.423582,-0.363019
3,0.756196,0.701106,0.055094,0.908833,0.937911,-0.060657,-0.473818,1.383407,0.878708,1.366116,...,0.010699,0.180546,-1.174782,0.044272,0.193422,-1.336530,-1.354113,-0.364440,-1.452520,-0.260586
4,0.745962,0.689887,0.072436,0.218246,0.942701,-0.102724,-0.469684,1.387666,0.884046,1.370650,...,0.043078,0.178003,-1.238520,0.070295,0.192018,-1.336415,-1.355173,-0.222580,-1.496741,-0.074345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4961,0.531052,0.512861,0.082841,0.203285,0.470491,-0.121457,-0.872193,0.884538,0.419886,0.876302,...,0.074210,-0.251754,-0.667906,0.113294,-0.242605,-1.813399,-1.831599,-0.222580,-2.177518,-0.139529
4962,0.536169,0.505381,0.082841,0.275288,0.470141,-0.175385,-0.872541,0.884476,0.419441,0.874654,...,0.095023,-0.254845,-1.217016,0.127890,-0.243436,-1.814552,-1.834389,-0.222580,-2.218325,-0.120905
4963,0.545124,0.494161,0.096715,0.562277,0.468155,-0.427428,-0.874284,0.884106,0.417383,0.869826,...,0.172906,-0.261627,-0.598766,0.165581,-0.247335,-1.820433,-1.844153,-0.222580,-0.143723,-0.074345
4964,0.546403,0.492914,0.055094,0.592084,0.467863,-0.652223,-0.874484,0.883982,0.417105,0.869296,...,0.175351,-0.261976,-0.518554,0.165762,-0.247699,-1.821009,-1.844432,-0.222580,-0.143723,-0.074345


In [16]:
X_train,X_test,y_train,y_test=train_test_split(X_new, y, test_size=0.1, random_state=1)

In [17]:
from sklearn.ensemble import AdaBoostClassifier

# base_model = DecisionTreeClassifier(max_depth=5)

start_time = time.time()

clf_4= RandomForestClassifier(n_estimators=100, random_state=43)
# create the AdaBoost classifier
clf_ada_dt = AdaBoostClassifier(base_estimator=clf_4)

# train the model using the training data
clf_ada_dt.fit(X_train, y_train)

# make predictions on the testing data
y_pred = clf_ada_dt.predict(X_test)

end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: {:.2f} seconds".format(elapsed_time))

# evaluate the model's accuracy
score = accuracy_score(y_test, y_pred)

model_precision.append(precision_score(y_test,y_pred,average='weighted'))
model_recal.append(recall_score(y_test,y_pred,average='weighted'))
model_f1.append(f1_score(y_test,y_pred,average='weighted'))
print(f'Accuracy: {round(score*100,2)}%')
model_acc.append(score)
model_time.append(elapsed_time)
model_nam.append("AdaBoost")

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Elapsed time: 2.93 seconds
Accuracy: 92.35%


In [18]:
data = {'Accuracy': model_acc, 'Precision': model_precision, 'Recall': model_recal, 'f1_score': model_f1}
df1 = pd.DataFrame(data)
# df1.to_csv('final_int'+str(number)+'.csv', index=False)

In [19]:
from sklearn.model_selection import cross_val_score
cv_score = cross_val_score(clf_ada_dt,X_new,y,cv=100)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 17 members, which is less than n_splits=100.
  warnings.warn(("The least populated class in y has only %d"
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:

KeyboardInterrupt: 

In [ ]:
cv_score

In [ ]:
cv_score.mean()